In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
names = ['UserID','MovieID','Rating','Timestamp']

In [4]:
raw = pd.read_table("aligned.txt",names = names, delim_whitespace = True)

In [5]:
raw.UserID = raw.UserID.astype(int)
raw.MovieID = raw.MovieID.astype(int)

In [6]:
def loss(mu,b_u,b_m,v_u,v_m, y_coo, lb,size):
    result = 0
    for i,j,v in zip(y_train_coo.row, y_train_coo.col, y_train_coo.data):
        result += (mu + b_u[i] + b_m[j] + np.dot(v_u[i],v_m[j]) - v)**2
    result = result/size
    result = result**(1/2)
    return result

In [333]:
loss(mu,b_u,b_m,v_u,v_m,y_train_coo,lb,size)

1.0523598341061313

In [7]:
aligned = raw.as_matrix()

In [8]:
aligned = aligned.astype('i8')

In [10]:
modified = np.zeros([cnt,aligned.shape[1]])

In [11]:
count = 0
for i in range(aligned.shape[0]):
    if(aligned[i,0]<1000 and aligned[i,1]<1000):
        modified[count] = aligned[i]
        count += 1

In [12]:
modified = modified.astype('i8')

In [9]:
cnt = 41893

In [14]:
train = modified[:(modified.shape[0]*4)//5,:]
test = modified[(modified.shape[0]*4)//5+1:,:]

In [15]:
n_movie = 1000
n_users = 1000

In [16]:
from scipy.sparse import lil_matrix
from scipy.sparse import csr_matrix

In [17]:
from scipy.sparse import coo_matrix

In [18]:
y_test = np.zeros((n_users, n_movie))
y_train = np.zeros((n_users, n_movie))

In [20]:
for i in range(train.shape[0]):
    y_train[train[i,0] - 1,train[i,1] - 1] = train[i,2]
for i in range(test.shape[0]):
    y_test[test[i,0] - 1,test[i,1] - 1] = test[i,2]

In [97]:
y_train_coo = coo_matrix(y_train)
y_test_coo = coo_matrix(y_test)

In [226]:
mu = np.mean(modified[:,2])
b_u = np.zeros(n_users)
b_m = np.zeros(n_movie)
v_u = np.random.randn(n_users,8)
v_m = np.random.randn(n_movie,8)
size = modified.shape[0]
eta = 0.001
lb = 0.03

In [40]:
def SGD(mu, b_u, b_m, v_u, v_m, y_train_coo, y_test_coo, size, lb,eta,step):
    temp_loss = loss(mu, b_u,b_m,v_u,v_m,y_test_coo,lb,size)
    for k in range(step):
        print(k)
        print(temp_loss)
        for i,j,v in zip(y_train_coo.row, y_train_coo.col, y_train_coo.data):
            rho = mu + b_u[i] + b_m[j] + np.dot(v_u[i],v_m[j]) - v
            b_u[i] -= eta*(rho + lb*b_u[i])
            b_m[j] -= eta*(rho + lb*b_m[j])
            v_u[i] -= eta*(rho*v_m[j] + lb*v_u[i])
            v_m[j] -= eta*(rho*v_u[i] + lb*v_m[j])
        temp_loss = loss(mu, b_u,b_m,v_u,v_m,y_test_coo,lb,size)

In [219]:
SGD(mu,b_u,b_m,v_u,v_m,y_train_coo,y_test_coo,size,lb,eta,30)

0
2.77041568475
1
2.27363427054
2
1.96512376901
3
1.7545908804
4
1.60191322352
5
1.48628402557
6
1.39579007585
7
1.32311567248
8
1.26352363357
9
1.21381420376
10
1.17174971847
11
1.13571896109
12
1.10453252239
13
1.07729331476
14
1.05331197983
15
1.03205005746
16
1.01308084986
17
0.996061871977
18
0.980715073271
19
0.966812386206
20
0.954164998228
21
0.942615273081
22
0.932030587787
23
0.922298575245
24
0.913323412057
25
0.905022893155
26
0.897326105354
27
0.890171561554
28
0.883505692647
29
0.877281619625


In [25]:
mnames = ['MovidID','Title','Genre']
movies = pd.read_table("movies.dat",delimiter = "::",names = mnames, nrows = 1000,engine = 'python',header = None)

In [26]:
movies_mtx = movies.as_matrix()

In [27]:
for i in range(1000):
    movies_mtx[i,2] = np.array(movies_mtx[i,2].split("|"))

In [28]:
labels = np.array([movies_mtx[i,2] for i in range(1000)])

In [29]:
from sklearn.preprocessing import LabelEncoder

In [139]:
lab = []
for i in range(1000):
    lab += labels[i].tolist()

In [140]:
enc = LabelEncoder()
enc.fit(lab)

LabelEncoder()

In [141]:
for i in range(1000):
    labels[i] = enc.transform(labels[i])

In [228]:
v_c = np.random.rand(8, enc.classes_.shape[0])

In [197]:
idx_mtx = np.zeros((n_movie, enc.classes_.shape[0]))

In [198]:
for i in range(1000):
    for j in range(labels[i].shape[0]):
        idx_mtx[i,labels[i][j] - 1] = 1

In [232]:
def mod_SGD(mu, b_u, b_m, v_u, v_m, y_train_coo, y_test_coo, size, lb,eta,step,v_c, idx_mtx):
    temp_loss = mod_loss(mu, b_u,b_m,v_u,v_m,y_test_coo,lb,size, v_c, idx_mtx)
    for k in range(step):
        print(k)
        print(temp_loss)
        for i,j,v in zip(y_train_coo.row, y_train_coo.col, y_train_coo.data):
            addition = np.dot(v_c,idx_mtx[j].reshape(19,1)).reshape(-1)
            index = np.array(idx_mtx[j].nonzero()).reshape(-1)
            rho = mu + b_u[i] + b_m[j] + np.dot(v_u[i],v_m[j] + addition) - v
            b_u[i] -= eta*(rho + lb*b_u[i])
            b_m[j] -= eta*(rho + lb*b_m[j])
            v_u[i] -= eta*(rho*(v_m[j] + addition) + lb*v_u[i])
            v_m[j] -= eta*(rho*v_u[i] + lb*v_m[j])
            for m in index:
                v_c[:,m] -= eta*(rho*v_u[i] + lb*v_c[:,m])
        temp_loss = loss(mu, b_u,b_m,v_u,v_m,y_test_coo,lb,size)

In [233]:
def mod_loss(mu,b_u,b_m,v_u,v_m, y_coo, lb,size, v_c, idx_mtx):
    result = 0
    for i,j,v in zip(y_train_coo.row, y_train_coo.col, y_train_coo.data):
        result += (mu + b_u[i] + b_m[j] + np.dot(v_u[i],v_m[j] + np.dot(v_c,idx_mtx[j].reshape(19,1)).reshape(-1)) - v)**2
    result = result/size
    result = result**(1/2)
    return result

In [235]:
mod_SGD(mu,b_u,b_m,v_u,v_m,y_train_coo,y_test_coo,size,lb,eta,30, v_c, idx_mtx)

0
0.873908178107
1
0.895578096497
2
0.890085334391
3
0.884904802698
4
0.880010659243
5
0.875379783543
6
0.870991430832
7
0.866826937315
8
0.862869468051
9
0.85910380044
10
0.855516137607
11
0.852093946987
12
0.84882582023
13
0.845701351236
14
0.842711029631
15
0.83984614747
16
0.83709871729
17
0.834461399942
18
0.831927440868
19
0.829490613699
20
0.827145170216
21
0.824885795853
22
0.82270757004
23
0.820605930793
24
0.818576643018
25
0.8166157701
26
0.814719648379
27
0.81288486418
28
0.811108233103
29
0.80938678133


In [236]:
pred_y = np.zeros((n_users, n_movie))

In [237]:
for i,j,v in zip(y_test_coo.row, y_test_coo.col, y_test_coo.data):
    pred_y[i][j] = (mu + b_u[i] + b_m[j] + np.dot(v_u[i],v_m[j] + np.dot(v_c,idx_mtx[j].reshape(19,1)).reshape(-1)))
for i,j,v in zip(y_train_coo.row, y_train_coo.col, y_train_coo.data):
    pred_y[i][j] = (mu + b_u[i] + b_m[j] + np.dot(v_u[i],v_m[j] + np.dot(v_c,idx_mtx[j].reshape(19,1)).reshape(-1)))

In [220]:
pred_old_y = np.zeros((n_users, n_movie))

In [221]:
for i,j,v in zip(y_test_coo.row, y_test_coo.col, y_test_coo.data):
    pred_old_y[i][j] = (mu + b_u[i] + b_m[j] + np.dot(v_u[i],v_m[j]))
for i,j,v in zip(y_train_coo.row, y_train_coo.col, y_train_coo.data):
    pred_old_y[i][j] = (mu + b_u[i] + b_m[j] + np.dot(v_u[i],v_m[j]))

In [183]:
num_labels = [labels[i].shape[0] for i in range(1000)]

In [184]:
num_labels

[5,
 3,
 2,
 3,
 1,
 3,
 2,
 2,
 1,
 3,
 3,
 2,
 2,
 1,
 3,
 2,
 3,
 3,
 1,
 5,
 3,
 5,
 3,
 2,
 2,
 1,
 1,
 2,
 5,
 2,
 1,
 2,
 3,
 4,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 3,
 1,
 3,
 3,
 2,
 4,
 4,
 2,
 3,
 3,
 3,
 2,
 2,
 1,
 3,
 1,
 3,
 2,
 3,
 2,
 1,
 2,
 2,
 1,
 3,
 1,
 2,
 1,
 4,
 1,
 2,
 2,
 2,
 2,
 3,
 1,
 1,
 2,
 2,
 3,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 3,
 2,
 3,
 3,
 2,
 1,
 2,
 1,
 2,
 4,
 1,
 3,
 1,
 2,
 2,
 4,
 1,
 1,
 3,
 3,
 4,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 3,
 1,
 1,
 3,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 3,
 2,
 4,
 2,
 4,
 1,
 3,
 2,
 2,
 2,
 3,
 4,
 3,
 1,
 3,
 4,
 3,
 3,
 1,
 3,
 3,
 4,
 2,
 3,
 3,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 3,
 1,
 3,
 2,
 1,
 3,
 2,
 1,
 2,
 1,
 1,
 2,
 3,
 2,
 1,
 6,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 3,
 1,
 1,
 1,
 4,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 3,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 3,
 1,
 1,
 2,
 3,
 2,
 2,
 4,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 5,
 1,
 2,
 2,
 1,
 2,


In [187]:
def mse(v1,v2):
    return (np.sum((v1 - v2)**2)/v1.shape[0])**(1/2)

In [286]:
errors_old = [mse(pred_old_y[:,i],y_test[:,i] + y_train[:,i]) for i in range(1000)]
errors_new = [mse(pred_y[:,i],y_test[:,i] + y_train[:,i]) for i in range(1000)]

In [288]:
datt_old = [[num_labels[i],errors_old[i]] for i in range(1000)]
datt_new = [[num_labels[i],errors_new[i]] for i in range(1000)]

In [289]:
datt_old = np.array(datt_old)
datt_new = np.array(datt_new)

In [290]:
datt_old = datt_old[datt_old[:,0].argsort()]
datt_new = datt_new[datt_new[:,0].argsort()]

In [291]:
datt_old = np.array(datt_old)
datt_new = np.array(datt_new)

In [292]:
bine_new = np.zeros((6,1000))
bine_old = np.zeros((6,1000))

In [293]:
count1 = 0
count2 = 0
count3 = 0
count4 = 0
count5 = 0
count6 = 0
for i in range(1000):
    if(datt_old[i,0] == 1.0):
        bine_old[0,count1] = datt_old[i,1]
        count1 += 1
    if(datt_old[i,0] == 2.0):
        bine_old[1,count2] = datt_old[i,1]
        count2 += 1
    if(datt_old[i,0] == 3.0):
        bine_old[2,count3] = datt_old[i,1]
        count3 += 1
    if(datt_old[i,0] == 4.0):
        bine_old[3,count4] = datt_old[i,1]
        count4 += 1
    if(datt_old[i,0] == 5.0):
        bine_old[4,count5] = datt_old[i,1]
        count5 += 1
    if(datt_old[i,0] == 6.0):
        bine_old[5,count6] = datt_old[i,1]
        count6 += 1

In [294]:
count1 = 0
count2 = 0
count3 = 0
count4 = 0
count5 = 0
count6 = 0
for i in range(1000):
    if(datt_new[i,0] == 1.0):
        bine_new[0,count1] = datt_new[i,1]
        count1 += 1
    if(datt_new[i,0] == 2.0):
        bine_new[1,count2] = datt_new[i,1]
        count2 += 1
    if(datt_new[i,0] == 3.0):
        bine_new[2,count3] = datt_new[i,1]
        count3 += 1
    if(datt_new[i,0] == 4.0):
        bine_new[3,count4] = datt_new[i,1]
        count4 += 1
    if(datt_new[i,0] == 5.0):
        bine_new[4,count5] = datt_new[i,1]
        count5 += 1
    if(datt_new[i,0] == 6.0):
        bine_new[5,count6] = datt_new[i,1]
        count6 += 1

In [295]:
mnn = np.mean(bine_new, axis = 1)
mnn

array([ 0.08014945,  0.07219575,  0.04094297,  0.01760891,  0.00693481,
        0.00102355])

In [296]:
mno = np.mean(bine_old, axis = 1)
mno

array([ 0.08437713,  0.07819788,  0.04263213,  0.01829819,  0.00705564,
        0.00087418])